In [19]:
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Votre base devra contenir ces différentes tables :

# Une table appelée population, contenant la population de chaque pays pour 2013. Elle devra contenir 4 colonnes : pays, code_pays, annee, population.
# Question 15 : Proposez une clé primaire pertinente pour cette table.


In [23]:
# Supression Chine sur dataset population 
POP = pd.read_csv('fr_population.csv')
POP.drop(33,axis=0, inplace=True)

# Projections 'Code zone', 'Zone', 'Valeur', 'Année' sur table population 
# Renomer les colonnes 
population = POP[['Code zone', 'Zone', 'Valeur', 'Année']].rename(columns={'Code zone':'code_pays','Zone':'pays','Valeur':'population','Année': 'annee'}).reset_index()[['code_pays','pays','population','annee']]
population

,code_pays,pays,population,annee
0,2,Afghanistan,30552,2013
1,202,Afrique du Sud,52776,2013
2,3,Albanie,3173,2013
3,4,Algérie,39208,2013
4,79,Allemagne,82727,2013
...,...,...,...,...
169,236,Venezuela (République bolivarienne du),30405,2013
170,237,Viet Nam,91680,2013
171,249,Yémen,24407,2013
172,251,Zambie,14539,2013


In [22]:
# Le choix de la clé primaire 
Clé_primaire = 'code_pays'
print("La clé primaire pour la table population sera : {Clé_primaire} ".format(Clé_primaire=Clé_primaire))

La clé primaire pour la table population sera : code_pays 


In [ ]:
population.to_csv("population.csv", index = False)

### Création table population dans MySQLworkbench

In [40]:
# Création table population (vide)
# Importation fichier population.csv pour avoir les données dans MySQLWorkbench

# CREATE TABLE `population` (
 # `code_pays` int DEFAULT NULL,
  #`pays` varchar(45) DEFAULT NULL,
  #`population` int DEFAULT NULL,
  #`annee` int DEFAULT NULL
#) ENGINE=InnoDB DEFAULT CHARSET=utf8


# Une table appelée dispo_alim contenant pour chaque pays et pour chaque produit en 2013, les informations suivantes:

## la nature du produit (deux valeurs possibles : “animal” ou “végétal”)
## disponibilité alimentaire en tonnes
## disponibilité alimentaire en Kcal/personne/jour
## disponibilité alimentaire de protéines en g/personne/jour
## disponibilité alimentaire de matières grasses en g/personne/jour
## Elle devra contenir ces colonnes : pays, code_pays, année, produit, code_produit, origin, dispo_alim_tonnes, dispo_alim_kcal_p_j, dispo_prot, dispo_mat_gr .
# Question 16 : Proposez une clé primaire pertinente pour cette table.

In [20]:
vegetaux = pd.read_csv("fr_vegetaux.csv")
vegetaux.drop(vegetaux.index[19971: 20831],0, inplace = True)

animaux = pd.read_csv("fr_animaux.csv")
animaux.drop(animaux.index[6985:7243],0,inplace=True)

# Création colonne Origin et modification des valeurs en animal sur table animaux 
# Projections
animaux['Origin']=animaux['Produit']
animaux['Origin']=animaux['Origin'].apply(lambda x: 'animal')
animal = animaux[['Code zone','Zone', 'Année', 'Produit', 'Code Produit','Élément', 'Valeur', 'Origin']].reset_index()[['Code zone','Zone', 'Année', 'Produit', 'Code Produit','Élément', 'Valeur', 'Origin']]


# Création colonne Origin et modification des valeurs en végétal sur table vegetaux 
# Projection
vegetaux['Origin']=vegetaux['Produit']
vegetaux['Origin']=vegetaux['Origin'].apply(lambda x: 'végétal')
vegetal = vegetaux[['Code zone','Zone', 'Année', 'Produit', 'Code Produit','Élément', 'Valeur', 'Origin']].reset_index()[['Code zone','Zone', 'Année', 'Produit', 'Code Produit','Élément', 'Valeur', 'Origin']]

# Concaténation entre animal et vegetal
dispo_alim1 = pd.concat([animal, vegetal], axis=0)
dispo_alim1.head()
dispo_alim1.tail()

# Restriction Nourriture','Disponibilité alimentaire (Kcal/personne/jour)','Disponibilité de protéines en quantité (g/personne/jour)','Disponibilité de matière grasse en quantité (g/personne/jour)
dispo_alim1 = dispo_alim1[(dispo_alim1['Élément'].isin(['Nourriture','Disponibilité alimentaire (Kcal/personne/jour)','Disponibilité de protéines en quantité (g/personne/jour)','Disponibilité de matière grasse en quantité (g/personne/jour)']))].reset_index()[['Code zone','Zone', 'Année', 'Produit', 'Code Produit','Élément', 'Valeur', 'Origin']]
dispo_alim1['Élément'].unique()


# Table pivot avec aggrégation sum
#fillna(0)
# rename_axis(None, axis=1)
dispo_alim = pd.pivot_table(dispo_alim1, values='Valeur', index=['Code zone','Zone','Année', 'Produit','Code Produit', 'Origin'],
                    columns=['Élément'], aggfunc=np.sum).reset_index().fillna(0).rename_axis(None, axis=1)

# Conversion colonne nourriture en tonnes  et renommer les colonnes 
dispo_alim['Nourriture'] = dispo_alim['Nourriture'].apply(lambda x : x*1000)
dispo_alim = dispo_alim.rename(columns={'Code zone': 'code_pays','Zone':'pays', 'Année': 'annee', 'Produit': 'produit', 'Code Produit':'code_produit','Origin':'origin', 'Disponibilité alimentaire (Kcal/personne/jour)': 'dispo_alim_kcal_p_j','Disponibilité de matière grasse en quantité (g/personne/jour)':'dispo_mat_gr', 'Disponibilité de protéines en quantité (g/personne/jour)': 'dispo_prot','Nourriture':'dispo_alim_tonnes' })
dispo_alim

,code_pays,pays,annee,produit,code_produit,origin,dispo_alim_kcal_p_j,dispo_mat_gr,dispo_prot,dispo_alim_tonnes
0,1,Arménie,2013,Abats Comestible,2736,animal,14.0,0.39,2.25,14000.0
1,1,Arménie,2013,"Agrumes, Autres",2614,végétal,0.0,0.00,0.01,1000.0
2,1,Arménie,2013,Aliments pour enfants,2680,végétal,2.0,0.01,0.07,0.0
3,1,Arménie,2013,Ananas,2618,végétal,0.0,0.00,0.00,0.0
4,1,Arménie,2013,Animaux Aquatiques Autre,2769,animal,0.0,0.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...
14255,276,Soudan,2013,Viande d'Ovins/Caprins,2732,animal,63.0,5.15,3.85,376000.0
14256,276,Soudan,2013,Viande de Bovins,2731,animal,64.0,4.96,4.46,409000.0
14257,276,Soudan,2013,Viande de Volailles,2734,animal,4.0,0.27,0.45,48000.0
14258,276,Soudan,2013,"Viande, Autre",2735,animal,22.0,1.55,1.94,194000.0


In [21]:
# Choix de la clé primaire 
Clé_primaire = 'Le couple code_pays et code_produit'
print("La clé primaire pour la table dispo_alim sera : {Clé_primaire} ".format(Clé_primaire=Clé_primaire))

La clé primaire pour la table dispo_alim sera : Le couple code_pays et code_produit 


In [ ]:
dispo_alim.to_csv("dispo_alim.csv", index = False, encoding='Utf-8')

### Création table dispo_alim dans MySQLWorkbench

In [41]:
# Création table dispo_alim (vide)
# Importation fichier dispo_alim.csv pour avoir les données dans MySQLWorkbench

 #CREATE TABLE `dispo_alim` (
 # `code_pays` int NOT NULL,
 # `pays` varchar(50) DEFAULT NULL,
 # `annee` smallint DEFAULT NULL,
 # `produit` varchar(50) DEFAULT NULL,
 # `code_produit` int NOT NULL,
  #`origin` varchar(50) DEFAULT NULL,
  #`dispo_alim_kcal_p_j` float DEFAULT NULL,
 # `dispo_mat_gr` float DEFAULT NULL,
 # `dispo_prot` float DEFAULT NULL,
  #`dispo_alim_tonnes` float DEFAULT NULL,
 # PRIMARY KEY (`code_pays`,`code_produit`)
#) ENGINE=InnoDB DEFAULT CHARSET=utf8


# Une table appelée equilibre_prod contenant pour chaque pays et pour chaque produit en 2013, les quantités suivantes :
### disponibilité intérieure
### aliments pour animaux
### semences
### pertes
### transformés
### nourriture
### autres utilisations
# Elle devra contenir ces colonnes : pays, code_pays, année, produit, code_produit, dispo_int, alim_ani, semences, pertes, transfo, nourriture, autres_utilisations.
## Question 17 : Proposez une clé primaire pertinente pour cette table.



In [25]:
# Projections colonnes sur table animaux et vegetaux 
# Concaténation entre animal1 et vegetaux1

animal1 = animaux[['Code zone', 'Zone', 'Année', 'Produit' ,'Code Produit','Élément', 'Valeur']]
vegetaux1 = vegetaux[['Code zone', 'Zone', 'Année', 'Produit' ,'Code Produit','Élément', 'Valeur']]
equilibre_prod1 = pd.concat([animal1,vegetaux1], axis=0).reset_index()[['Code zone', 'Zone', 'Année', 'Produit' ,'Code Produit','Élément', 'Valeur']]
equilibre_prod1.head()

# Restriction dans colonne Élément pour les valeurs qui vont devenir mes colonnes 
Equilibre_prod2 = equilibre_prod1[(equilibre_prod1['Élément'].isin(['Disponibilité intérieure','Aliments pour animaux','Semences','Pertes' ,'Traitement', 'Nourriture','Autres utilisations (non alimentaire)']))].reset_index()[['Code zone', 'Zone', 'Année', 'Produit' ,'Code Produit','Élément', 'Valeur']]

# Table pivot avec aggrégation sum 
equilibre_prod = pd.pivot_table(Equilibre_prod2, values='Valeur', index=['Code zone','Zone','Année', 'Produit','Code Produit'],
                    columns=['Élément'], aggfunc=np.sum).reset_index().fillna(0).rename_axis(None, axis=1)

# Renommer les colonnes 
equilibre_prod = equilibre_prod.rename(columns={'Code zone': 'code_pays','Zone':'pays', 'Année': 'annee', 'Produit': 'produit', 'Code Produit': 'code_produit','Aliments pour animaux':'alim_ani','Autres utilisations (non alimentaire)':'autres_utilisations', 'Disponibilité intérieure': 'dispo_int', 'Nourriture':'nourriture','Pertes': 'pertes','Semences':'semences','Traitement':'transfo'})  
equilibre_prod                           


,code_pays,pays,annee,produit,code_produit,alim_ani,autres_utilisations,dispo_int,nourriture,pertes,semences,transfo
0,1,Arménie,2013,Abats Comestible,2736,0.0,3.0,17.0,14.0,0.0,0.0,0.0
1,1,Arménie,2013,"Agrumes, Autres",2614,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2,1,Arménie,2013,"Alcool, non Comestible",2659,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,Arménie,2013,Aliments pour enfants,2680,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,Arménie,2013,Ananas,2618,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
15377,276,Soudan,2013,Viande de Suides,2733,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15378,276,Soudan,2013,Viande de Volailles,2734,0.0,0.0,48.0,48.0,0.0,0.0,0.0
15379,276,Soudan,2013,"Viande, Autre",2735,0.0,0.0,194.0,194.0,0.0,0.0,0.0
15380,276,Soudan,2013,Vin,2655,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
# Choix de la clé primaire 
Clé_primaire = 'Le couple code_pays et code_produit'
print("La clé primaire pour la table equilibre_prod sera : {Clé_primaire} ".format(Clé_primaire=Clé_primaire))


La clé primaire pour la table equilibre_prod sera : Le couple code_pays et code_produit 


In [ ]:
equilibre_prod.to_csv("equilibre_prod.csv", index = False)

### Création table equilibre_prod dans MySQLWorkbench

In [43]:
# Création table equilibre_prod (vide)
# Importation fichier equilibre_prod.csv pour avoir les données dans MySQLWorkbench

#CREATE TABLE `equilibre_prod` (
  #`code_pays` int NOT NULL,
  #`pays` varchar(50) DEFAULT NULL,
  #`annee` int DEFAULT NULL,
  #`produit` varchar(50) DEFAULT NULL,
  #`code_produit` int NOT NULL,
  #`alim_ani` double DEFAULT NULL,
 # `autres_utilisations` double DEFAULT NULL,
  #`dispo_int` double DEFAULT NULL,
 # `nourriture` double DEFAULT NULL,
  #`pertes` double DEFAULT NULL,
 # `semences` double DEFAULT NULL,
 # `transfo` double DEFAULT NULL,
  #PRIMARY KEY (`code_pays`,`code_produit`)
#) ENGINE=InnoDB DEFAULT CHARSET=utf8


# Une table appelée sous_nutrition, contenant le nombre de personnes en sous-alimentation pour #chaque pays en 2013. Elle devra contenir 4 colonnes : pays, code_pays, année, nb_personnes.
# Question 18 : Vous vous en doutez... proposez encore une fois une clé primaire pertinente pour cette table !

In [33]:
# Téléchargement fichier prévalence issu de la banque mondiale 
# Supprimer colonne Chine pour fichier sousalimentation.csv
# Reperer les valeurs uniques de la colonne description du symbole 
SousAlimentation=pd.read_csv('fr_sousalimentation.csv')
prevalence=pd.read_csv('prévalence.csv')
SousAlimentation1 = SousAlimentation[SousAlimentation['Zone']!='Chine']
SousAlimentation['Description du Symbole'].unique()

# Restriction des valeurs pour Estimation FAO
sous_nutrition = SousAlimentation1[SousAlimentation1['Description du Symbole']=='Estimation FAO']

#  Restriction des valeurs pour Estimation FAO et égale à <0.1
pays_string = sous_nutrition[sous_nutrition['Valeur']== '<0.1']

# Restriction des valeurs pour Estimation FAO différents de <0.1
pays_sous_nutrition = sous_nutrition[sous_nutrition['Valeur']!= '<0.1']


# Projections colonnes 'Code zone', 'Zone', 'Année', 'Valeur'
# Restriction année 2013
# Conversion colonne Valeur en float 
# Création colonne 'nb_personnes'en millions 
Pays_sous_nutrition1 = pays_sous_nutrition[['Code zone', 'Zone', 'Année', 'Valeur']]
Pays_sous_nutrition1 = Pays_sous_nutrition1[Pays_sous_nutrition1['Année']=='2012-2014'].reset_index()[['Code zone', 'Zone', 'Année', 'Valeur']]
Pays_sous_nutrition1['Valeur']= Pays_sous_nutrition1['Valeur'].astype(float)
Pays_sous_nutrition1['nb_personnes']= Pays_sous_nutrition1['Valeur']*1000000
Pays_sous_nutrition2 = Pays_sous_nutrition1[['Code zone', 'Zone', 'Année', 'nb_personnes']]
Pays_sous_nutrition2

# Restriction année 2013 pour les pays en sous nutrition (<0.1)
# Projections colonnes 'Code zone', 'Zone' , 'Année', 'Valeur'
pays_string1 = pays_string[pays_string['Année']=='2012-2014']
pays_string2 = pays_string1[['Code zone', 'Zone' , 'Année', 'Valeur']].reset_index()[['Code zone', 'Zone' , 'Année', 'Valeur']]
pays_string2.head()

# Création colonne 'nb_personnes' avec les valeurs de la questions 10
# Supprimer colonne Valeur 
pays_string2['nb_personnes']=[12224.728, 24545.765, 10919.367, 72668.216, 70439.384, 61768.764,  4402.992, 40857.907, 36355.536, 66817.080, 73130.880, 3452.480, 91346.948, 46130.823, 70484.568, 18982.800, 11083.448, 6191.454, 5530.793, 23242.527, 44877.862,44877.862, 16681.275]
pays_string3 = pays_string2.drop(['Valeur'], axis=1)
pays_string3.head()

########################### La table sous_nutrition #########################
# Concaténation entre Pays_sous_nutrition2 et pays_string3
# renommer les colonnes
sous_nutrition = pd.concat([Pays_sous_nutrition2 , pays_string3], axis=0).reset_index()[['Code zone','Zone', 'Année','nb_personnes']]
sous_nutrition = sous_nutrition.rename(columns={'Code zone':'code_pays','Année':'annee','Zone':'pays'})
sous_nutrition

,code_pays,pays,annee,nb_personnes
0,2,Afghanistan,2012-2014,7900000.000
1,202,Afrique du Sud,2012-2014,2600000.000
2,3,Albanie,2012-2014,200000.000
3,4,Algérie,2012-2014,1700000.000
4,7,Angola,2012-2014,8100000.000
...,...,...,...,...
114,244,Samoa,2012-2014,5530.793
115,193,Sao Tomé-et-Principe,2012-2014,23242.527
116,207,Suriname,2012-2014,44877.862
117,220,Trinité-et-Tobago,2012-2014,44877.862


In [34]:
# Le choix de la clé primaire id_code_pays
Clé_primaire = 'code_pays'
print("La clé primaire pour la table sous_nutrition sera : {Clé_primaire} ".format(Clé_primaire=Clé_primaire))


La clé primaire pour la table sous_nutrition sera : code_pays 


In [ ]:
sous_nutrition.to_csv("sous_nutrition.csv", index = False)

### Création table sous_nutrition dans MySQLWorkbench

In [38]:
# Création table sous_nutrition (vide)
# Importation fichier sous_nutrition.csv pour avoir les données dans MySQLWorkbench


#CREATE TABLE `sous_nutrition` (
 # `code_pays` int NOT NULL,
 # `pays` varchar(50) DEFAULT NULL,
  #`annee` varchar(20) DEFAULT NULL,
 # `nb_personnes` double DEFAULT NULL,
  #PRIMARY KEY (`code_pays`)
#) ENGINE=InnoDB DEFAULT CHARSET=utf8


# Question 19 : Écrivez les requêtes SQL permettant de connaître…

In [45]:
print()
print("Create Connexion")

import pymysql.cursors

connection = pymysql.connect(host='localhost',
                             user='root',
                             password='Louca221217',
                             db='openclassrooms_P3',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

print ("connection")
 


Create Connexion
connection


# Les 10 pays ayant le plus haut ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant

In [46]:
with connection.cursor() as cursor:
    sql = """SELECT pays, SUM(dispo_prot*365/1000) AS 'ratio dispo_alim/habitant (proteine kg)'
          FROM dispo_alim
          GROUP BY  pays
          ORDER BY SUM(dispo_prot*365/1000)DESC
          LIMIT 10 ;"""
    
    cursor.execute(sql)
    print ("cursor.description: ", cursor.description)
 
     

cursor.description:  (('pays', 253, None, 200, 200, 0, True), ('ratio dispo_alim/habitant (proteine kg)', 5, None, 23, 23, 31, True))


In [47]:
# dataframe de la requête SQL ci-dessus
data1 = [row for row in cursor]
pd.DataFrame(data1)

,pays,ratio dispo_alim/habitant (proteine kg)
0,Islande,48.566900
1,Chine - RAS de Hong-Kong,47.110550
2,Israël,46.720000
3,Lituanie,45.391400
4,Maldives,44.646800
5,Finlande,42.909400
6,Luxembourg,41.478600
7,Monténégro,40.843500
8,Pays-Bas,40.682900
9,Albanie,40.650049


# Les 10 pays ayant le plus haut ratio disponibilité alimentaire/habitant en termes de en termes de kcal par habitant.¶

In [48]:
with connection.cursor() as cursor:
    sql = """SELECT  pays, SUM(dispo_alim_kcal_p_j*365) AS 'ratio dispo_alim/habitant (kcal)'
          FROM dispo_alim
          GROUP BY pays
          ORDER BY SUM(dispo_alim_kcal_p_j*365) DESC
          LIMIT 10;"""
    cursor.execute(sql)
    print ("cursor.description: ", cursor.description)
 
                 

cursor.description:  (('pays', 253, None, 200, 200, 0, True), ('ratio dispo_alim/habitant (kcal)', 5, None, 23, 23, 31, True))


In [49]:
# dataframe de la requête SQL ci-dessus
data2 = [row for row in cursor]
pd.DataFrame(data2)     

,pays,ratio dispo_alim/habitant (kcal)
0,Autriche,1376050.0
1,Belgique,1364005.0
2,Turquie,1353420.0
3,États-Unis d'Amérique,1343930.0
4,Israël,1317650.0
5,Irlande,1314730.0
6,Italie,1305970.0
7,Luxembourg,1292100.0
8,Égypte,1284070.0
9,Allemagne,1278595.0


# Pour l'année 2013,# les 10 pays ayant le plus faible ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant.

In [50]:
with connection.cursor() as cursor:
    sql = """SELECT  pays, SUM(dispo_prot*365/1000) AS 'ratio dispo_alim/habitant'
          FROM dispo_alim
          GROUP BY pays
          ORDER BY SUM(dispo_prot*365/1000)ASC
          LIMIT 10; """
    
    cursor.execute(sql)
    print ("cursor.description: ", cursor.description)
 
         

cursor.description:  (('pays', 253, None, 200, 200, 0, True), ('ratio dispo_alim/habitant', 5, None, 23, 23, 31, True))


In [51]:
# dataframe de la requête SQL ci-dessus
data3 = [row for row in cursor]
pd.DataFrame(data3)

,pays,ratio dispo_alim/habitant
0,Libéria,13.74590
1,Guinée-Bissau,16.07825
2,Mozambique,16.67320
3,République centrafricaine,16.80460
4,Madagascar,17.04185
5,Haïti,17.41050
6,Zimbabwe,17.63680
7,Congo,18.76465
8,Ouganda,19.21360
9,Sao Tomé-et-Principe,19.38150


# La quantité totale (en kg) de produits perdus par pays en 2013.

In [54]:
with connection.cursor() as cursor:
    sql = """SELECT pays, SUM(pertes*1000000) AS 'pertes (kg)'
    FROM equilibre_prod
    GROUP BY pays 
    ORDER BY SUM(pertes*1000000) DESC
    ;"""
    
    cursor.execute(sql)
    print ("cursor.description: ", cursor.description)
 

cursor.description:  (('pays', 253, None, 200, 200, 0, True), ('pertes (kg)', 5, None, 23, 23, 31, True))


In [55]:
# dataframe de la requête SQL ci-dessus
data4 = [row for row in cursor]
pd.DataFrame(data4)

,pays,pertes (kg)
0,"Chine, continentale",8.957500e+10
1,Brésil,7.591400e+10
2,Inde,5.593000e+10
3,Nigéria,1.985400e+10
4,Indonésie,1.308100e+10
...,...,...
169,Bermudes,0.000000e+00
170,Kiribati,0.000000e+00
171,Islande,0.000000e+00
172,Maldives,0.000000e+00


# Les 10 pays pour lesquels la proportion de personnes sous-alimentées est la plus forte

In [57]:
with connection.cursor() as cursor:
    sql = """SELECT p.pays, (s.nb_personnes/(p.population*1000))*100 AS 'propor_sous_alimentées'
          FROM sous_nutrition as s
          INNER JOIN population as p ON s.pays = p.pays
          ORDER BY (s.nb_personnes/(p.population*1000))*100 DESC
          LIMIT 10;"""
    
    cursor.execute(sql)
    print ("cursor.description: ", cursor.description)
 

cursor.description:  (('pays', 253, None, 180, 180, 0, True), ('propor_sous_alimentées', 5, None, 23, 23, 31, True))


In [58]:
# dataframe de la requête SQL ci-dessus
data5 = [row for row in cursor]
pd.DataFrame(data5)

,pays,propor_sous_alimentées
0,Haïti,50.402249
1,Zambie,48.146365
2,Zimbabwe,46.643110
3,République centrafricaine,43.327556
4,République populaire démocratique de Corée,42.578831
5,Congo,40.467626
6,Tchad,38.206628
7,Angola,37.723547
8,Libéria,37.261295
9,Madagascar,35.768811


# Les 10 produits pour lesquels le ratio Autres utilisations/Disponibilité intérieure est le plus élevé.

In [59]:
with connection.cursor() as cursor:
    sql = """SELECT produit, SUM(autres_utilisations) / SUM(dispo_int) AS 'ratio (ratio Autres utilisations/Disponibilité intérieure)'
          FROM equilibre_prod
          GROUP BY Produit
          ORDER BY SUM(autres_utilisations) / SUM(dispo_int) DESC
          LIMIT 10;"""
 
 
    cursor.execute(sql)
    print ("cursor.description: ", cursor.description)
    

cursor.description:  (('produit', 253, None, 200, 200, 0, True), ('ratio (ratio Autres utilisations/Disponibilité intérieure)', 5, None, 23, 23, 31, True))


/Users/stephaneea/opt/anaconda3/lib/python3.7/site-packages/pymysql/cursors.py:329: Warning: (1365, 'Division by 0')
  self._do_get_result()


In [60]:
# dataframe de la requête SQL ci-dessus
data6 = [row for row in cursor]
pd.DataFrame(data6)

,produit,ratio (ratio Autres utilisations/Disponibilité intérieure)
0,"Alcool, non Comestible",1.001288
1,Huil Plantes Oleif Autr,0.754744
2,Huile de Palmistes,0.703957
3,Huile de Palme,0.698109
4,Girofles,0.646154
5,Huile de Colza&Moutarde,0.551290
6,Graisses Animales Crue,0.470219
7,Huiles de Poissons,0.448068
8,Huile de Soja,0.416172
9,Plantes Aquatiques,0.379299


# Question 20 : pour quelques uns des produits identifiés dans cette dernière requête SQL, supposez quelles sont ces "autres utilisations" possibles (recherchez sur internet !)

In [ ]:
####################################### Alcool, non Comestible ##############################################

Vous trouverez l’alcool dénaturé dans l’alcool réactif pour une utilisation dans les laboratoires,  
désinfectant pour les mains , l’ alcool à friction, et le carburant pour les lampes à alcool. 
Elle se trouve également dans les produits cosmétiques , parfum et d’autres produits de soins personnels.
(source: wikipedia)


In [ ]:
####################################### Huile de palme #################################################

À l'échelle mondiale, l'huile de palme est principalement utilisée dans l'industrie alimentaire (près de 70 %),
dans l'oléochimie (savon, cosmétiques…) et en tant que agrocarburant (plus de 20 %).
La majeure partie de la production part donc en direction de l’agroalimentaire : 
on retrouve principalement cet ingrédient dans les produits cuisinés de nos supermarchés.
    
Ainsi de nombreux gâteaux apéritifs, biscuits sucrés, céréales pour petit déjeuner, plats préparés (surgelés ou non)
en contiennent.
Le secteur des cosmétiques utilise aussi en masse cet ingrédient pour produire nos produits de beauté et d'hygiène. 
L'oléochimie comprend aussi la fabrication de peintures, lubrifiants et laques, jadis issus du pétrole,
que l'on peut aujourd'hui 
produire à partir de dérivés de l'huile de palme.
(source: wikipedia)


In [ ]:
######################################## GRAISSE ANIMALE ####################################################

1)Base du savon
Les graisses animales entrent dans la composition de certains savons. 
Elles sont alors une source d'acide oléique (acide cis-9-octadécénoïque).

La graisse de bœuf (Adeps bovis) est transformée par saponification en tallowate de sodium et le saindoux
(graisse de porc) en lardate de sodium 
(également appelé Adeps suillus selon l'INCI) par association avec de la soude (hydroxyde de sodium).
                                                                                                                                                  
2) Cosmétique et parfumerie                                                                                                                                                 

In [ ]:
######################################### MANIOC ###############################################################


Savez-vous que l’on peut aussi utiliser cette racine tubéreuse pour fabriquer des boissons alcoolisées ? 
Comme la bière ou l’eau-de-vie, …

Pour illustrer cela, au Brésil, il existe la Tiquira, le Cachaça, le Cauim, le Cavim, …

Par ailleurs, on peut même en tirer du sirop de glucose.

Ensuite, avec son amidon, on peut confectionner du papier et, même de la colle.

Encore ensuite, les feuilles, et même parfois les tiges peuvent servir à fabriquer du sel alimentaire.
Par ailleurs, avec la racine, elles font aussi une excellente source de nourriture pour le bétail.

Et enfin, pour finir, on peut faire du combustible avec le bois.